<span>
<img src="http://ash.readthedocs.io/en/latest/_static/ash.png" width="260px" align="right"/>
</span>
<span>
<b>Author:</b> <a href="https://andreafailla.github.io">Andrea Failla</a><br/>
<b>Python version:</b>  3.9<br/>
<b>ASH version:</b>  0.1.0<br/>
<b>Last update:</b> July 2025
</span>

<a id="attributed-stream-hypergraphs-ash"></a>
# Attributed Stream Hypergraphs (ASH)

In [ ]:
#!pip install -e ../

<a id="table-of-contents"></a>
# Table of Contents

- [Introduction](#introduction)
- [Hyperedge-Level Measures](#hyperedge-level-measures)
  - [Purity of Hyperedge Profiles](#purity-of-hyperedge-profiles)
  - [Entropy of Hyperedge Profiles](#entropy-of-hyperedge-profiles)
- [Star-Level Measures](#star-level-measures)
  - [Star Profile Entropy](#star-profile-entropy)
  - [Star Profile Homogeneity](#star-profile-homogeneity)
- [Group-Level Measures: Average Group Degree](#group-level-measures-average-group-degree)
- [Temporal Consistency of Attributes](#temporal-consistency-of-attributes)


<a id="introduction"></a>
## Introduction

This tutorial will guide you through some functionalities offered by ASH to analyze properties related to node attributes.

All methods in this module share the following characteristics:
1. They return results for all attributes;
2. Their return type is a dictionary, typically containing the attribute name (or value) and the corresponding score;
3. Imternally, they process <code>NProfile</code> objects. If you are not familiar with them, see the corresponding section in the "basics" tutorial notebook.


[🔝 To top](#table-of-contents)

In [ ]:
# Import necessary libraries
import networkx as nx
from ash_model.utils import from_networkx_maximal_cliques_list
from ash_model.measures import (
    hyperedge_profile_purity,
    hyperedge_profile_entropy,
    star_profile_entropy,
    star_profile_homogeneity,
    average_group_degree,
    attribute_consistency
)

In [12]:
import random
snapshots = []
for _ in range(10):
    g = nx.barabasi_albert_graph(100, 5)
    for node in g.nodes:
        g.nodes[node]['color'] = random.choice(['red', 'blue', 'green', 'yellow'])
        g.nodes[node]['role'] = random.choice(['admin', 'user', 'guest'])
        g.nodes[node]['team'] = random.choice(['engineering', 'marketing', 'sales'])
    snapshots.append(g)
ash = from_networkx_maximal_cliques_list(snapshots)

In [ ]:
# the maximal cliques method preserves the attributes
ash.list_node_attributes()

{'color': {'blue', 'green', 'red', 'yellow'},
 'role': {'admin', 'guest', 'user'},
 'team': {'engineering', 'marketing', 'sales'}}

## 2. Hyperedge-Level Measures

### 2.1 Purity of Hyperedge Profiles

Purity measures the dominance of the most frequent attribute value among
the nodes in a hyperedge. A purity score of 1.0 means all nodes share
the same attribute value.

In [15]:
# Example: compute purity for hyperedge 'e42' at time tid=0
purity = hyperedge_profile_purity(ash, hyperedge_id='e1', tid=0)
print("Purity per attribute:", purity)


Purity per attribute: {'color': {'red': 0.6666666666666666}, 'role': {'user': 0.6666666666666666}, 'team': {'marketing': 1.0}}


### 2.2 Entropy of Hyperedge Profiles

Entropy quantifies the diversity of attribute values. Higher entropy
indicates more variety.



In [16]:
entropy = hyperedge_profile_entropy(ash, hyperedge_id='e1', tid=0)
print("Entropy per attribute:", entropy)


Entropy per attribute: {'color': np.float64(0.9182958340544896), 'role': np.float64(0.9182958340544896), 'team': 0}


## 3. Star-Level Measures

The **star** of a node is the set of hyperedges in which it
participates. We can analyze attribute distributions across its
neighborhood.

### 3.1 Star Profile Entropy

Compute attribute entropy across either the aggregated hyperedge
profiles (`method='aggregate'`) or directly across neighbors
(`method='collapse'`).

In [17]:
# Using aggregate method
star_ent_agg = star_profile_entropy(ash, node_id=10, tid=0, method='aggregate')
print("Star entropy (aggregate):", star_ent_agg)

# Using collapse method
star_ent_col = star_profile_entropy(ash, node_id=10, tid=0, method='collapse')
print("Star entropy (collapse):", star_ent_col)

Star entropy (aggregate): {'color': np.float64(0.7313776131322518), 'role': np.float64(0.9435794406736076), 'team': np.float64(0.8783471047618533)}
Star entropy (collapse): {'color': np.float64(0.9773429734731779), 'role': np.float64(0.991532399729883), 'team': np.float64(1.0)}


### 3.2 Star Profile Homogeneity

Homogeneity measures how aligned the neighbors are with the focal node’s
attribute values.

In [23]:
star_homog = star_profile_homogeneity(ash, node_id=10, tid=0, method='aggregate')
print("Star homogeneity:", star_homog)

Star homogeneity: {'color': 0.5555555555555556, 'role': 0.2777777777777778, 'team': 0.4444444444444444}


## 4. Group-Level Measures: Average Group Degree

This function computes, for each categorical attribute, the average
hyperedge-degree of nodes sharing each attribute value.



In [20]:
avg_deg = average_group_degree(ash, tid=0, hyperedge_size=None)
for attr, group in avg_deg.items():
    print(f"Attribute '{attr}':")
    for value, mean_deg in group.items():
        print(f"  {value}: {mean_deg:.2f}")

Attribute 'color':
  green: 10.67
  yellow: 7.38
  red: 8.19
  blue: 10.11
Attribute 'role':
  user: 10.41
  admin: 8.95
  guest: 7.58
Attribute 'team':
  marketing: 10.65
  engineering: 8.81
  sales: 7.34


## 5. Temporal Consistency of Attributes

Consistency measures how stable a node’s attribute values remain over
time (1.0 = never changes).

In [22]:
# For a single node:
consistency_node_5 = attribute_consistency(ash, node=5)
print("Node 5 consistency:", consistency_node_5)

# For all nodes:
all_consistency = attribute_consistency(ash)
# Example: show first 3 nodes
for n, cons in list(all_consistency.items())[:3]:
    print(f"Node {n}: {cons}")

Node 5 consistency: {'color': np.float64(0.039035952556318865), 'role': np.float64(0.41832813428211324), 'team': np.float64(0.06276943678387048)}
Node 0: {'color': np.float64(0.07678032766449228), 'role': np.float64(0.039770282139238944), 'team': np.float64(0.14132728892674506)}
Node 98: {'color': np.float64(0.07678032766449228), 'role': np.float64(0.14132728892674506), 'team': np.float64(0.039770282139238944)}
Node 3: {'color': np.float64(0.07678032766449228), 'role': np.float64(0.27015330083790257), 'team': np.float64(0.039770282139238944)}
